In [ ]:
# Importieren der erforderlichen Bibliotheken
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, expr, collect_list, size, array_intersect, percent_rank, abs
)
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Spark Session initialisieren
spark = SparkSession.builder \
    .appName("BookCrossing_ALS") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()

In [ ]:
# Einlesen des Ratings-Datensatzes (mit delimiter ";")
ratings1 = spark.read.option("header", "true").option("delimiter", ";").csv("BX-Book-Ratings.csv")

# Skaliere die Book-Rating-Spalte auf eine 5er-Skala
ratings1 = ratings1.withColumn("Book-Rating", col("Book-Rating") / 2.0)

# Entferne fehlende Werte und Duplikate
ratings1 = ratings1.dropna().dropDuplicates()

# Damit die Nutzer-IDs aus beiden Datensätzen zusammengeführt werden können,
# benenne die ursprüngliche Nutzer-Spalte in Datensatz 1 um
ratings1 = ratings1.withColumnRenamed("User-ID", "userId_orig")
# Zeige 20 Einträge aus Datensatz 1
ratings1.show(20)

In [ ]:
# Einlesen des Datensatzes nach Angriff (tausche Angriffsart und Angriffsgröße jeweils aus)
ratings2 = spark.read.option("header", "true").csv("all_ratings_BookCrossing_reverse_bandwagon_20.0.csv")
# Entferne fehlende Werte und Duplikate
ratings2 = ratings2.dropna().dropDuplicates()

# Skaliere die Rating-Spalte auf eine 5er-Skala
ratings2 = ratings2.withColumn("rating", col("rating") / 2.0)

# Benenne die Nutzer-Spalte um, damit beide Datensätze denselben Namen haben
ratings2 = ratings2.withColumnRenamed("userId", "userId_orig")
# Zeige 20 Einträge aus Datensatz 2
ratings2.show(20)

In [ ]:
# Kombiniere alle eindeutigen Nutzer-IDs aus beiden Datensätzen
combined_users = ratings1.select("userId_orig").union(ratings2.select("userId_orig")).distinct()

# Fitte einen StringIndexer für die Nutzer auf dem kombinierten Datensatz
user_indexer = StringIndexer(inputCol="userId_orig", outputCol="userId", handleInvalid="skip")
user_indexer_model = user_indexer.fit(combined_users)

# Transformiere beide Datensätze mit dem gleichen Nutzer-Indexer
ratings1 = user_indexer_model.transform(ratings1)
ratings2 = user_indexer_model.transform(ratings2)

# Für Items: Beide Datensätze besitzen die Spalte "ISBN", also:
combined_items = ratings1.select("ISBN").union(ratings2.select("ISBN")).distinct()
item_indexer = StringIndexer(inputCol="ISBN", outputCol="ISBN_new", handleInvalid="skip")
item_indexer_model = item_indexer.fit(combined_items)

# Wende den Item-Indexer auf beide Datensätze an
ratings1 = item_indexer_model.transform(ratings1)
ratings2 = item_indexer_model.transform(ratings2)

# Für Datensatz 1: Indexierung der Ratings (Spalte "Book-Rating"); Ergebnis in "rating"
rating_indexer1 = StringIndexer(inputCol="Book-Rating", outputCol="rating", handleInvalid="skip")
ratings1 = rating_indexer1.fit(ratings1).transform(ratings1)

# Für Datensatz 2: Indexierung der Ratings (Spalte "rating"); Ergebnis in "rating_new"
rating_indexer2 = StringIndexer(inputCol="rating", outputCol="rating_new", handleInvalid="skip")
ratings2 = rating_indexer2.fit(ratings2).transform(ratings2)

# Entferne in Datensatz 1 unnötige Spalten
ratings1 = ratings1.drop('userId_orig', 'Book-Rating', 'ISBN')
ratings1.printSchema()

# Entferne in Datensatz 2 die ursprünglichen Spalten (z. B. "userId_orig", "rating", "Label", "ISBN")
ratings2 = ratings2.drop('userId_orig', 'rating', 'Label', 'ISBN')
ratings2.printSchema()

In [ ]:
# Teile die Daten in Trainings- und Testdaten auf in einem 80/20-Verhältnis
train1, test1 = ratings1.randomSplit([0.8, 0.2], seed=42)
print(f"Datensatz 1: Trainingsset: {train1.count()} Einträge, Testset: {test1.count()} Einträge")

train2, test2 = ratings2.randomSplit([0.8, 0.2], seed=42)
print(f"Datensatz 2: Trainingsset: {train2.count()} Einträge, Testset: {test2.count()} Einträge")

In [ ]:
# ALS-Algorithmus für Datensatz 1 (vor Angriff)
als1 = ALS(
    rank=100,
    maxIter=20,
    regParam=0.05,
    userCol='userId',
    itemCol='ISBN_new',
    ratingCol='rating',
    coldStartStrategy="drop"
)
als_model1 = als1.fit(train1)
predictions1 = als_model1.transform(test1)

In [ ]:
# RMSE und MAE für Datensatz 1 berechnen
evaluator_rmse1 = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
evaluator_mae1 = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
rmse1 = evaluator_rmse1.evaluate(predictions1)
mae1 = evaluator_mae1.evaluate(predictions1)

print(f"Datensatz 1 - RMSE: {rmse1}")
print(f"Datensatz 1 - MAE: {mae1}")

In [ ]:
# HitRate-Berechnung für Datensatz 1
window_spec1 = Window.orderBy(col("rating").desc())
test1 = test1.withColumn("percentile", percent_rank().over(window_spec1))
# Verwende die besten 20% der Bewertungen als relevante Items und gruppiere nach userId
relevant_items1 = test1.filter(col("percentile") <= 0.2) \
                       .groupBy("userId") \
                       .agg(expr("collect_list(ISBN_new) as relevant_items"))

# Verwende 10 und 20 als Werte für k, um die HitRate@10 sowie HitRate@20 zu berechnen
k_values = [10, 20]
for k in k_values:
    top_k_recommendations1 = als_model1.recommendForAllUsers(k)
    top_k_recommendations_exploded1 = top_k_recommendations1.withColumn(
        "recommended_item_ids",
        expr("transform(recommendations, x -> x['ISBN_new'])")
    )
    joined_data1 = top_k_recommendations_exploded1.join(relevant_items1, on="userId", how="inner")
    hit_data1 = joined_data1.withColumn(
        "relevant_in_recommendations",
        size(array_intersect(col("recommended_item_ids"), col("relevant_items")))
    )
    hit_data1 = hit_data1.withColumn("is_hit", col("relevant_in_recommendations") > 0)
    count_joined1 = joined_data1.count()
# Berechne die durchschnittliche HitRate über alle User
    hit_rate1 = hit_data1.filter(col("is_hit") == True).count() / count_joined1 if count_joined1 > 0 else None
    print(f"Datensatz 1 - HitRate@{k}: {hit_rate1}")

In [ ]:
# ALS-Algorithmus für Datensatz 2 (nach Angriff)
als2 = ALS(
    rank=100,
    maxIter=20,
    regParam=0.05,
    userCol='userId',
    itemCol='ISBN_new',
    ratingCol='rating_new',
    coldStartStrategy="drop"
)
als_model2 = als2.fit(train2)
predictions2 = als_model2.transform(test2)

In [ ]:
# RMSE und MAE für Datensatz 2 berechnen
evaluator_rmse2 = RegressionEvaluator(metricName="rmse", labelCol="rating_new", predictionCol="prediction")
evaluator_mae2 = RegressionEvaluator(metricName="mae", labelCol="rating_new", predictionCol="prediction")
rmse2 = evaluator_rmse2.evaluate(predictions2)
mae2 = evaluator_mae2.evaluate(predictions2)

print(f"Datensatz 2 - RMSE: {rmse2}")
print(f"Datensatz 2 - MAE: {mae2}")

In [ ]:
# HitRate-Berechnung für Datensatz 2
window_spec2 = Window.orderBy(col("rating_new").desc())
test2 = test2.withColumn("percentile", percent_rank().over(window_spec2))
# Verwende die besten 20% der Bewertungen als relevante Items und gruppiere nach userId
relevant_items2 = test2.filter(col("percentile") <= 0.2) \
                       .groupBy("userId") \
                       .agg(expr("collect_list(ISBN_new) as relevant_items"))

for k in k_values:
    top_k_recommendations2 = als_model2.recommendForAllUsers(k)
    top_k_recommendations_exploded2 = top_k_recommendations2.withColumn(
        "recommended_item_ids",
        expr("transform(recommendations, x -> x['ISBN_new'])")
    )
    joined_data2 = top_k_recommendations_exploded2.join(relevant_items2, on="userId", how="inner")
    hit_data2 = joined_data2.withColumn(
        "relevant_in_recommendations",
        size(array_intersect(col("recommended_item_ids"), col("relevant_items")))
    )
    hit_data2 = hit_data2.withColumn("is_hit", col("relevant_in_recommendations") > 0)
    count_joined2 = joined_data2.count()
# Berechne die durchschnittliche HitRate über alle User
    hit_rate2 = hit_data2.filter(col("is_hit") == True).count() / count_joined2 if count_joined2 > 0 else None
    print(f"Datensatz 2 - HitRate@{k}: {hit_rate2}")

In [ ]:
# Berechne Vorhersagen auf den Trainingsdaten beider Modelle
train_predictions1 = als_model1.transform(train1).select("userId", "ISBN_new", col("prediction").alias("prediction_1"))
train_predictions2 = als_model2.transform(train2).select("userId", "ISBN_new", col("prediction").alias("prediction_2"))

# Join der Vorhersagen über die gemeinsamen Schlüssel
common_predictions = train_predictions1.join(train_predictions2, on=["userId", "ISBN_new"], how="inner")

# Berechne den absoluten Unterschied und den durchschnittlichen Prediction Shift
common_predictions = common_predictions.withColumn("abs_diff", abs(col("prediction_1") - col("prediction_2")))
prediction_shift = common_predictions.agg({"abs_diff": "avg"}).collect()[0][0]
print(f"Prediction Shift: {prediction_shift}")

In [ ]:
# Beende die Spark-Session
spark.stop()